In [1]:
%cd '/Users/max/Projects/Coreference/'

/Users/max/Projects/Coreference


In [2]:
%cd 'rucoref'
from anaphoralib.corpora import rueval
from anaphoralib.tagsets import multeast
from anaphoralib.experiments.base import BaseClassifier
from anaphoralib import utils
%cd '..'

/Users/max/Projects/Coreference/rucoref


/Users/max/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


/Users/max/Projects/Coreference


In [3]:
import collections
import itertools

In [4]:
texts_dir = 'Corpus-2015/Tokens.train.txt.parsed'
gs_dir = 'Corpus-2015/Groups.train.txt'

tagset = multeast

random_state = 42

In [5]:
rucoref = rueval.RuCorefCorpus(multeast, rueval)

In [6]:
rucoref.load_texts(texts_dir)

In [13]:
for i, text in enumerate(rucoref.texts):
    print i
    find_groups_syntax(text, tagset, rucoref.parses[i])

0
время:время(Ncnsan, 3)
своих:свой(P---pga, 9)
прогулок:прогулка(Ncfpgn, 15)
окрестностях:окрестность(Ncfpln, 26)
Симеиза:симеиза(Ncmsgn, 39)
я:я(P-1-snn, 47)
внимание:внимание(Ncnsan, 57)
дачу:дача(Ncfsan, 78)
склоне:склон(Ncmsln, 103)
горы:гора(Ncfpan, 110)
даче:дача(Ncfsln, 123)
она:она(P-3fsnn, 166)
забором:забор(Ncmsin, 192)
калиткой:калитка(Ncfsin, 223)
которая:который(P--fsna, 233)
куста:куст(Ncmsgn, 275)
зелени:зелень(Ncfsgn, 281)
дерева:дерево(Ncnsgn, 292)
забором:забор(Ncmsin, 316)
дачи:дача(Ncfsgn, 332)
уступы:уступ(Ncmpnn, 343)
скал:скала(Ncfpgn, 361)
ними:они(P-3-pin, 371)
можжевельники:можжевельники(Ncmpny, 398)
сосны:сосна(Ncfpnn, 441)
фантазия:фантазия(Ncfsnn, 456)
голову:голова(Ncfsan, 482)
утесе:утес(Ncmsln, 521)
я:я(P-1-snn, 567)
дачи:дача(Ncfsgn, 583)
Я:я(P-1-snn, 589)
Любопытство:любопытство(Ncnsnn, 646)
я:я(P-1-snn, 683)
двор:двор(Ncmsan, 718)
жилища:жилище(Ncnsgn, 737)
скалы:скала(Ncfpan, 772)
дача:дача(Ncfsnn, 782)
я:я(P-1-snn, 824)
я:я(P-1-snn, 838)
угол:угол(

ValueError: 815 is not in list

In [7]:
pronouns = {u"его", u"ее", u"её", u"ей", u"ему", u"ею", u"им", u"ими", u"их", 
            u"которая", u"которого", u"которое", u"которой", u"котором", u"которому", u"которую", 
            u"которые", u"который", u"которым", u"которыми", u"которых", 
            u"него", u"нее", u"неё", u"ней", u"нем", u"нём", u"нему", u"нею", u"ним", u"ними", u"них", 
            u"он", u"она", u"они", u"оно", u"я",
            u"свое", u"своё", u"своего", u"своей", u"своем", u"своём", u"своему", 
            u"своею", u"свой", u"свои", u"своим", u"своими", u"своих", u"свою", u"своя", u"себе", u"себя", u"собой", u"собою"}

In [11]:
def is_np_head(word):
    return word.tag.startswith('N') or word.wordform[0].lower() in pronouns

In [9]:
def build_dependency_index(corpus, i_text):
    dependencies = collections.defaultdict(set)
    for w2_ind, word2 in enumerate(rucoref.texts[i_text]):
        w1_ind = corpus.parses_[i_text][w2_ind][0] - 1
        rel = corpus.parses_[i_text][w2_ind][1]
        word1 = corpus.texts[i_text][w1_ind]
        #print u' '.join(word1.wordform), u'->', u' '.join(word2.wordform) \
        #    if word1_index != -1 else 'ROOT', u'({})'.format(rel)
        if w1_ind != -1:
            dependencies[w1_ind].add((w2_ind, rel))
    return dependencies

In [10]:
def is_np_dependency(text, i, j, rel):
    w1 = text[i]
    w2 = text[j]
    
    res = True
    
    # filtering participials
    res &= not tagset.pos_filters['verb'](w2)
    
    # filtering interjections and other useless parts of speech
    res &= not (tagset.pos_filters['interjection'](w1) or tagset.pos_filters['interjection'](w2))
    res &= not (tagset.pos_filters['conj'](w2) and rel in (u'сравнит', u'примыкат'))
    res &= not (w2.tag == 'Q' and rel in (u'огранич',))
    
    # filtering bad roles
    res &= not rel == u'предик'
    
    return res

In [12]:
def find_groups_syntax(text, tagset, parses):
    nps = []

    def get_dependencies_recursive(i):
        res = {i}
        if i in dep_ind:
            for j in dep_ind[i]:
                if tagset.is_np_dependency(text[i], text[j[0]], j[1]):
                    res |= get_dependencies_recursive(j[0])
        return res

    def build_dependency_index():
        dependencies = collections.defaultdict(set)
        for w2_ind, word2 in enumerate(text):
            w1_ind = parses[w2_ind][0] - 1
            rel = parses[w2_ind][1]

            if w1_ind != -1:
                dependencies[w1_ind].add((w2_ind, rel))
        return dependencies

    dep_ind = build_dependency_index()
    for i, word in enumerate(text):
        if tagset.is_np_head(word):
            print word
            np_words = sorted(get_dependencies_recursive(i))
            while np_words and tagset.pos_filters['punctuation'](text[np_words[-1]]):
                np_words.pop()
            np_head = np_words.index(i)
            np_words = [text[w] for w in np_words]
            nps.append(utils.Group(wordform=list(itertools.chain.from_iterable(w.wordform for w in np_words)),
                             lemma=list(itertools.chain.from_iterable(w.lemma for w in np_words)),
                             tag=np_words[np_head].tag,
                             tags=[w.tag for w in np_words],
                             prob=1.0,
                             offset=np_words[0].offset,
                             length=np_words[-1].offset + np_words[-1].length - np_words[0].offset,
                             words=np_words[:],
                             head=np_head,
                             type='agr'
                             ))
    return nps

In [14]:
find_groups_syntax(rucoref.texts[0], multeast, rucoref.parses[0])

[время своих прогулок в окрестностях Симеиза(Ncnsan, 3),
 своих(P---pga, 9),
 своих прогулок в окрестностях Симеиза(Ncfpgn, 9),
 окрестностях Симеиза(Ncfpln, 26),
 Симеиза(Ncmsgn, 39),
 я(P-1-snn, 47),
 внимание на одинокую дачу(Ncnsan, 57),
 одинокую дачу(Ncfsan, 69),
 крутом склоне(Ncmsln, 96),
 горы(Ncfpan, 110),
 этой даче(Ncfsln, 118),
 она(P-3fsnn, 166),
 высоким забором(Ncmsin, 184),
 единственной низкой калиткой(Ncfsin, 203),
 которая(P--fsna, 233),
 куста зелени , ни дерева(Ncmsgn, 275),
 зелени(Ncfsgn, 281),
 дерева(Ncnsgn, 292),
 забором(Ncmsin, 316),
 дачи(Ncfsgn, 332),
 Кругом дачи - голые уступы желтоватых скал(Ncmpnn, 325),
 желтоватых скал(Ncfpgn, 350),
 ними(P-3-pin, 371),
 чахлые можжевельники и низкорослые , кривые горные сосны(Ncmpny, 391),
 низкорослые , кривые горные сосны(Ncfpnn, 414),
 фантазия(Ncfsnn, 456),
 голову(Ncfsan, 482),
 этом диком , голом утесе(Ncmsln, 503),
 я(P-1-snn, 567),
 дачи(Ncfsgn, 583),
 Я(P-1-snn, 589),
 Любопытство мое(Ncnsnn, 646),
 я(P-1-

In [11]:
def get_groups(corpus, i_text):
    nps = []
    
    def get_dependencies_recursive(i):
        res = set([i])
        if i in dep_ind:
            for j in dep_ind[i]:
                if is_np_dependency(corpus.texts[i_text], i, j[0], j[1]):
                    res |= get_dependencies_recursive(j[0])
        return res
    
    dep_ind = build_dependency_index(rucoref, i_text)
    for i, word in enumerate(corpus.texts[i_text]):
        if is_np_head(word):
            np_words = sorted(get_dependencies_recursive(i))
            while np_words and tagset.pos_filters['punctuation'](corpus.texts[i_text][np_words[-1]]):
                np_words.pop()
            np_head = np_words.index(i)
            np_words = [corpus.texts[i_text][w] for w in np_words]
            nps.append(utils.Group(wordform=list(itertools.chain.from_iterable(w.wordform for w in np_words)),
                                   lemma=list(itertools.chain.from_iterable(w.lemma for w in np_words)),
                                   tag=np_words[np_head].tag,
                                   tags=[w.tag for w in np_words],
                                   prob=1.0,
                                   offset=np_words[0].offset,
                                   length=np_words[-1].offset + np_words[-1].length - np_words[0].offset,
                                   words=np_words[:],
                                   head=np_head,
                                   type='agr'
                                  ))
    return nps

In [12]:
groups = get_groups(rucoref, 0)

In [13]:
groups[5].words[groups[5].head]

я(P-1-snn, 47)

In [15]:
groups

[время своих прогулок в окрестностях Симеиза(Ncnsan, 3),
 своих(P---pga, 9),
 своих прогулок в окрестностях Симеиза(Ncfpgn, 9),
 окрестностях Симеиза(Ncfpln, 26),
 Симеиза(Ncmsgn, 39),
 я(P-1-snn, 47),
 внимание на одинокую дачу(Ncnsan, 57),
 одинокую дачу(Ncfsan, 69),
 крутом склоне(Ncmsln, 96),
 горы(Ncfpan, 110),
 этой даче(Ncfsln, 118),
 она(P-3fsnn, 166),
 высоким забором(Ncmsin, 184),
 единственной низкой калиткой(Ncfsin, 203),
 которая(P--fsna, 233),
 куста зелени , ни дерева(Ncmsgn, 275),
 зелени(Ncfsgn, 281),
 дерева(Ncnsgn, 292),
 забором(Ncmsin, 316),
 дачи(Ncfsgn, 332),
 Кругом дачи - голые уступы желтоватых скал(Ncmpnn, 325),
 желтоватых скал(Ncfpgn, 350),
 ними(P-3-pin, 371),
 чахлые можжевельники и низкорослые , кривые горные сосны(Ncmpny, 391),
 низкорослые , кривые горные сосны(Ncfpnn, 414),
 фантазия(Ncfsnn, 456),
 голову(Ncfsan, 482),
 этом диком , голом утесе(Ncmsln, 503),
 я(P-1-snn, 567),
 дачи(Ncfsgn, 583),
 Я(P-1-snn, 589),
 Любопытство мое(Ncnsnn, 646),
 я(P-1-

In [16]:
rucoref.texts[0][7]

я(P-1-snn, 47)

In [17]:
print rucoref.parses_[0][7][1]

предик


In [18]:
groups[5]

я(P-1-snn, 47)

In [19]:
rucoref.create_indices()

Способ извлечения синтаксических ролей

In [20]:
print rucoref.parses_[0][rucoref.words_index[0][groups[5].words[groups[5].head].offset]][1]

предик
